# Sequence data preprocessing

In [1]:
import pandas as pd
import cudf
import pyreadr
import numpy as np
import torch
import pickle
from os import listdir
from os.path import isfile, join

In [11]:
# file1 = pickle.load(open('../SeqModel/SeqChunks/seq_data_1.sav', 'rb'))
file1.columns

Index(['patid', 'length_read_code_seq', 'read_code_seq_padded',
       'read_code_seq_padded_end', 'month_padded', 'month_padded_end'],
      dtype='object')

In [22]:
patient = pyreadr.read_r('../ServerData_13Oct2020/d_patient_overall.Rdata')
practice = pyreadr.read_r('../ServerData_13Oct2020/d_practice.Rdata')

/opt/conda/envs/rapids/lib/python3.10/site-packages/pyreadr/_pyreadr_parser.py:233: RuntimeWarning: invalid value encountered in cast
  df[colname] = df[colname].values.astype("datetime64[D]").astype(datetime)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in cast
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pandas/core/tools/datetimes.py:557: RuntimeWarning: invalid value encountered in multiply
  arr, tz_parsed = tslib.array_with_unit_to_datetime(arg, unit, errors=errors)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pyreadr/_pyreadr_parser.py:233: RuntimeWarning: invalid value encountered in cast
  df[colname] = df[colname].values.astype("datetime64[D]").astype(datetime)
/opt/conda/envs/rapids/lib/python3.10/site-packages/pyreadr/_pyreadr_parser.py:233: RuntimeWarning: invalid value encountered in cast
  df[col

In [25]:
patient = patient['d_patient_overall']
practice = practice['d_practice']

#Age in 2016-01-01
patient['age'] = patient.year_of_birth.apply(lambda x: 2016-x)

In [46]:
def fix_system(x):
    if ('SystmOne' in x):
        return 'SystemOne'
    elif ('emis' in x) | ('Emis' in x) | ('EMIS' in x):
        return 'EMIS'
    elif ('Vision' in x):
        return 'Vision'
    elif ('iSoft' in x):
        return 'iSoft'
    elif ('Microtest' in x):
        return 'Microtest'
    else:
        return 'unknown'

In [48]:
practice['system'] = practice.apply(lambda x: fix_system(x.system), axis=1)

In [50]:
patient = patient[['patid', 'practice_id', 'age']].merge(practice[['practice_id', 'Country', 'system']], how='left', on='practice_id')

In [ ]:
# Outcomes data
outcomes = cudf.read_csv("../FinalData/cleaned_outcomes_2vs1_15112023.csv")
outcomes['3months'] = outcomes.apply(lambda x: x.outcome_3months, axis=1)
outcomes['6months'] = outcomes.apply(lambda x: (x.outcome_3months)|(x.outcome_6months), axis=1)
outcomes['9months'] = outcomes.apply(lambda x: (x.outcome_3months)|(x.outcome_6months)|(x.outcome_9months), axis=1)
outcomes['12months'] = outcomes.apply(lambda x: (x.outcome_3months)|(x.outcome_6months)|(x.outcome_9months)|(x.outcome_12months), axis=1)

# Clinical

In [ ]:
#load clinical information
path = '../ServerData_13Oct2020/'
clinical_files = [join(path, f) for f in listdir(path) if (isfile(join(path, f))) & ('f_clinical_part' in f)]
clinical = pyreadr.read_r('../ServerData_13Oct2020/f_clinical_part1.Rdata')
clinical = clinical['f_clinical_part']
# for file in clinical_files[1:3]:
#     temp = pyreadr.read_r(file)
#     temp = temp['f_clinical_part']
#     print(temp.shape)
#     clinical = pd.concat([clinical, temp])
#     clinical.reset_index(drop=True, inplace=True)

In [ ]:
clinical.shape

In [ ]:
#data selection 
clinical = clinical.dropna(subset=['code_id'])
clinical['event_date'] = pd.to_datetime(clinical['event_date'])
clinical = clinical.loc[(clinical['event_date'] >= '2013-01-01') & (clinical['event_date'] < '2018-01-01')]

In [ ]:
print(clinical.patid.unique().shape)
print(clinical.shape)

In [ ]:
#vocab and code2idx generation
vocab = clinical.code_id.unique().tolist()
idx = range(1, len(vocab)+1)

code2idx = dict(zip(vocab, idx))
idx2code = dict(zip(idx, vocab))

code2idx['PAD'] = 0
idx2code[0] = 'PAD'
VOCAB_SIZE = len(code2idx)
print('code2idx Size: {}'.format(len(code2idx)))
print('idx2code Size: {}'.format(len(idx2code)))

In [ ]:
#extract year, month, day from event date
clinical['day'] = clinical.apply(lambda x: str(x['event_date'].day), axis=1)
clinical['month'] = clinical.apply(lambda x: str(x['event_date'].month), axis=1)
clinical['year'] = clinical.apply(lambda x: str(x['event_date'].year), axis=1)

In [ ]:
print(clinical.patid.unique().shape)
print(clinical.shape)

In [ ]:
clinical.reset_index(drop=True, inplace=True)
clinical['read_code_seq_perdate'] = clinical.sort_values(['event_date'], ascending=True).groupby(['patid', 'event_date'])['code_id'].transform(lambda x: ', '.join(x))
clinical['read_code_seq_perdate'] = clinical['read_code_seq_perdate'].apply(lambda x: 'start_visit, ' + x + ', end_visit')
clinical = clinical.drop_duplicates(subset=['patid', 'event_date']).reset_index(drop=True)
clinical['read_code_seq'] = clinical.sort_values(['event_date'], ascending=True).groupby(['patid'])['read_code_seq_perdate'].transform(lambda x: ', '.join(x))
# clinical['read_code_seq'] = clinical.sort_values(['event_date'], ascending=True).groupby('patid')['code_id'].transform(lambda x: ', '.join(x))
event_data_seq = clinical.sort_values(['event_date']).groupby('patid').agg({'day': lambda x: x.tolist(),
                                                          'month': lambda x: x.tolist(),
                                                          'year': lambda x: x.tolist()}).reset_index()
clinical = clinical.drop_duplicates(subset=['patid']).reset_index(drop=True)     

In [ ]:
print(clinical.patid.unique().shape)
print(clinical.shape)

In [ ]:
data = clinical[['patid', 'read_code_seq']].merge(event_data_seq, how='left', on='patid')
data = data.merge(outcomes[['patid', 'outcome_3months','outcome_combined_6months','outcome_combined_12months',
                           'outcome_combined_24months']], how='inner', on='patid')
data.outcome_combined_12months.value_counts()

In [ ]:
print(data.patid.unique().shape)
print(data.shape)

In [ ]:
data['read_code_seq'] = data['read_code_seq'].apply(lambda x: x.strip('""').split(', '))
data['length_read_code_seq'] = data['read_code_seq'].apply(lambda x: len(x))
data = data[data.length_read_code_seq > 10]

In [ ]:
data_therapy[data_therapy.length_read_code_seq>50].shape

In [ ]:
print(data.patid.unique().shape)
print(data.shape)

In [ ]:
#padding at the beginning of the list
max_seq = 500
def make_uniform_data(x):
    if len(x) < max_seq:
        pads = ['PAD'] * (max_seq - len(x))
        return pads + x
    elif len(x) > max_seq:
        x = x[len(x)-max_seq:]
        return x
    else:
        return x

In [ ]:
data['read_code_seq_padded'] = data['read_code_seq'].apply(lambda x: make_uniform_data(x))
data['read_code_seq_padded_idx'] = data['read_code_seq_padded'].apply(lambda x: [code2idx.get(key) for key in x])
data = data.merge(patient[['patid', 'Country', 'age']], how='left', on='patid')
data = data.drop_duplicates(subset=['patid']).reset_index(drop=True)     

In [ ]:
print(data.patid.unique().shape)
print(data.shape)

In [ ]:
#SAVE all important materials
pickle.dump(code2idx, open('../SeqModel/code2idx.sav', 'wb'))
pickle.dump(idx2code, open('../SeqModel/idx2code.sav', 'wb'))
pickle.dump(data, open('../SeqModel/data.sav', 'wb'))
data.to_csv('../SeqModel/seqDataRaw.csv', index_label=False, index=False)

In [ ]:
data.length_read_code_seq.hist(bins=50)

In [ ]:
data[data.length_read_code_seq > 300].shape

In [ ]:
# a = pickle.load(open('../SeqModel/data.sav', 'rb'))

# Medication

In [ ]:
#add paractice information

#load clinical information
path = '../ServerData_13Oct2020/'
therapy_files = [join(path, f) for f in listdir(path) if (isfile(join(path, f))) & ('f_therapy_part' in f)]
therapy = pyreadr.read_r('../ServerData_13Oct2020/f_therapy_part1.Rdata')
therapy = therapy['f_therapy_part']
# for file in therapy_files[1:3]:
#     temp = pyreadr.read_r(file)
#     temp = temp['f_therapy_part']
#     print(temp.shape)
#     therapy = pd.concat([therapy, temp])
#     therapy.reset_index(drop=True, inplace=True)

In [ ]:
#data selection 
therapy = therapy.dropna(subset=['code_id'])
therapy['event_date'] = pd.to_datetime(therapy['event_date'])
therapy = therapy.loc[(therapy['event_date'] >= '2013-01-01') & (therapy['event_date'] < '2018-01-01')]

In [ ]:
# therapy['new_code'] = therapy.apply(lambda x: x['code_id'] + '_' + str(x['dose_id']), axis=1)

In [ ]:
therapy.shape

In [ ]:
alllist = c['read_code_seq'].values[0]+c['read_code_seq'].values[0]

In [ ]:
len(list(set(alllist)))

In [ ]:
therapy.patid.unique().shape

In [ ]:
therapy.shape

In [ ]:
#vocab and code2idx generation
vocab_therapy = therapy.code_id.unique().tolist()
idx_therapy = range(1, len(vocab_therapy)+1)

code2idx_therapy = dict(zip(vocab_therapy, idx_therapy))
idx2code_therapy = dict(zip(idx_therapy, vocab_therapy))

code2idx_therapy['PAD'] = 0
# code2idx_therapy['start_visit'] = 1
# code2idx_therapy['end_visit'] = 2
idx2code_therapy[0] = 'PAD'
# idx2code_therapy[1] = 'start_visit'
# idx2code_therapy[2] = 'end_visit'
VOCAB_SIZE = len(code2idx_therapy)
print('code2idx Size: {}'.format(len(code2idx_therapy)))
print('idx2code Size: {}'.format(len(idx2code_therapy)))
# print(code2idx)


#vocab for month sequence
month_vocab_therapy = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
month_idx_therapy = range(1, len(month_vocab_therapy)+1)

month2idx_therapy = dict(zip(month_vocab_therapy, month_idx_therapy))
idx2month_therapy = dict(zip(month_idx_therapy, month_vocab_therapy))

month2idx_therapy['PAD'] = 0
# code2idx_all['start_visit'] = 1
# code2idx_all['end_visit'] = 2
idx2month_therapy[0] = 'PAD'
# idx2code_all[1] = 'start_visit'
# idx2code_all[2] = 'end_visit'
# VOCAB_SIZE = len(month2idx_all)
print('code2idx Size: {}'.format(len(month2idx_therapy)))
# print('idx2code Size: {}'.format(len(idx2code_all)))



In [ ]:
#extract year, month, day from event date
therapy['day'] = therapy.apply(lambda x: str(x['event_date'].day), axis=1)
therapy['month'] = therapy.apply(lambda x: str(x['event_date'].month), axis=1)
therapy['year'] = therapy.apply(lambda x: str(x['event_date'].year), axis=1)
therapy_event_date_seq = therapy.sort_values(['event_date'],  ascending=False).groupby('patid').agg({'day': lambda x: x.tolist(),
                                                          'month': lambda x: x.tolist(),
                                                          'year': lambda x: x.tolist()}).reset_index()

In [ ]:
# therapy['read_code_seq_perdate'] = therapy.sort_values(['event_date'], ascending=True).groupby(['patid', 'event_date'])['code_id'].transform(lambda x: ', '.join(x))
# therapy['read_code_seq_perdate'] = therapy['read_code_seq_perdate'].apply(lambda x: 'start_visit, ' + x + ', end_visit')
# therapy = therapy.drop_duplicates(subset=['patid', 'event_date']).reset_index(drop=True)
# therapy['read_code_seq'] = therapy.sort_values(['event_date'], ascending=True).groupby(['patid'])['read_code_seq_perdate'].transform(lambda x: ', '.join(x))
# therapy['read_code_seq'] = therapy.sort_values(['event_date'], ascending=True).groupby(['patid'])['code_id'].transform(lambda x: ', '.join(x))
therapy['read_code_seq'] = therapy.sort_values(['event_date'], ascending=False).groupby('patid')['code_id'].transform(lambda x: ', '.join(x))
therapy = therapy.drop_duplicates(subset=['patid']).reset_index(drop=True)     
therapy.reset_index(drop=True, inplace=True)
therapy = therapy.drop_duplicates(subset=['patid']).reset_index(drop=True)     

In [ ]:
therapy.patid.unique().shape

In [ ]:
therapy.shape

In [ ]:
therapy.shape

In [ ]:
data_therapy = therapy[['patid', 'read_code_seq']].merge(therapy_event_date_seq, how='left',  on='patid')
print(data_therapy.shape)
data_therapy = data_therapy.merge(outcomes[['patid', '3months','6months', '9months', '12months',]], how='inner', on='patid')
print(data_therapy.shape)
# data_therapy.outcome_combined_12months.value_counts()

In [ ]:
data_therapy['read_code_seq'] = data_therapy['read_code_seq'].apply(lambda x: x.strip('""').split(', '))
data_therapy['length_read_code_seq'] = data_therapy['read_code_seq'].apply(lambda x: len(x))
# data_therapy = data_therapy[data_therapy.length_read_code_seq > 10]

In [ ]:
data_therapy.patid.unique().shape
data_therapy.shape

In [ ]:
data_therapy.length_read_code_seq.describe()

In [ ]:
data_therapy[data_therapy.length_read_code_seq > 150].shape

In [ ]:
#padding at the beginning of the list

def make_uniform_data(x):
    max_seq = 150
    if len(x) < max_seq:
        pads = ['PAD'] * (max_seq - len(x))
        return pads + x
    elif len(x) > max_seq:
        x = x[len(x)-max_seq:]
        return x
    else:
        return x
    
#padding at the end of the list
def make_uniform_data_end(x):
    max_seq = 150
    if len(x) < max_seq:
        pads = ['PAD'] * (max_seq - len(x))
        return x + pads
    elif len(x) > max_seq:
        x = x[len(x)-max_seq:]
        return x
    else:
        return x

In [ ]:
data_therapy['read_code_seq_padded'] = data_therapy['read_code_seq'].apply(lambda x: make_uniform_data(x))
data_therapy['read_code_seq_padded_end'] = data_therapy['read_code_seq'].apply(lambda x: make_uniform_data_end(x))
data_therapy['month_padded'] = data_therapy['month'].apply(lambda x: make_uniform_data(x))
data_therapy['month_padded_end'] = data_therapy['month'].apply(lambda x: make_uniform_data_end(x))

In [ ]:
# padded to index, and merge with patient data (age, Country)
# data_all_big = data_all
data_therapy['read_code_seq_padded_idx'] = data_therapy['read_code_seq_padded'].apply(lambda x: [code2idx_therapy.get(key) for key in x])
data_therapy['read_code_seq_padded_idx_end'] = data_therapy['read_code_seq_padded_end'].apply(lambda x: [code2idx_therapy.get(key) for key in x])
data_therapy['month_padded_idx'] = data_therapy['month_padded'].apply(lambda x: [month2idx_therapy.get(key) for key in x])
data_therapy['month_padded_idx_end'] = data_therapy['month_padded_end'].apply(lambda x: [month2idx_therapy.get(key) for key in x])

In [ ]:
data_therapy = data_therapy.merge(patient[['patid', 'Country', 'age']], how='left', on='patid')
data_therapy = data_therapy.drop_duplicates(subset=['patid']).reset_index(drop=True)     

In [ ]:
print(data_therapy.patid.unique().shape)
print(data_therapy.shape)

In [ ]:
#SAVE all important materials
pickle.dump(code2idx_therapy, open('../SeqModel/code2idx_therapy_long22112023.sav', 'wb'))
pickle.dump(idx2code_therapy, open('../SeqModel/idx2code_therapy_long22112023.sav', 'wb'))
pickle.dump(month_idx_therapy, open('../SeqModel/month2idx_therapy_long22112023.sav', 'wb'))
pickle.dump(idx2month_therapy, open('../SeqModel/idx2month_therapy_long22112023.sav', 'wb'))
pickle.dump(data_therapy, open('../SeqModel/data_therapy_long22112023.sav', 'wb'))
# data_therapy.to_csv('../SeqModel/seqData_therapyRaw.csv', index_label=False, index=False)

In [ ]:
data_therapy.columns

In [ ]:
data_therapy.length_read_code_seq.hist()

In [ ]:
data_therapy[data_therapy.length_read_code_seq<50].shape

In [ ]:
data_therapy.shape

# Combine clinical + therapy

In [ ]:
#load clinical information
path = '../ServerData_13Oct2020/'
clinical_files = [join(path, f) for f in listdir(path) if (isfile(join(path, f))) & ('f_clinical_part' in f)]
clinical = pyreadr.read_r('../ServerData_13Oct2020/f_clinical_part1.Rdata')
clinical = clinical['f_clinical_part']
for file in clinical_files[1:3]:
    temp = pyreadr.read_r(file)
    temp = temp['f_clinical_part']
    print(temp.shape)
    clinical = pd.concat([clinical, temp])
    clinical.reset_index(drop=True, inplace=True)
    
#data selection 
clinical = clinical.dropna(subset=['code_id'])
clinical['event_date'] = pd.to_datetime(clinical['event_date'])
clinical = clinical.loc[(clinical['event_date'] >= '2008-01-01') & (clinical['event_date'] < '2018-01-01')]
clinical = clinical[['patid', 'event_date', 'code_id']]

In [ ]:
#add paractice information

#load clinical information
path = '../ServerData_13Oct2020/'
therapy_files = [join(path, f) for f in listdir(path) if (isfile(join(path, f))) & ('f_therapy_part' in f)]
therapy = pyreadr.read_r('../ServerData_13Oct2020/f_therapy_part1.Rdata')
therapy = therapy['f_therapy_part']
for file in therapy_files[1:3]:
    temp = pyreadr.read_r(file)
    temp = temp['f_therapy_part']
    print(temp.shape)
    therapy = pd.concat([therapy, temp])
    therapy.reset_index(drop=True, inplace=True)
    
#data selection 
therapy = therapy.dropna(subset=['code_id'])
therapy['event_date'] = pd.to_datetime(therapy['event_date'])
therapy = therapy.loc[(therapy['event_date'] >= '2008-01-01') & (therapy['event_date'] < '2018-01-01')]
therapy = therapy[['patid', 'event_date', 'code_id']]

In [ ]:
clinical['read_code_seq_perdate'] = clinical.sort_values(['event_date'], ascending=True).groupby(['patid', 'event_date'])['code_id'].transform(lambda x: ', '.join(x))
therapy['read_code_seq_perdate'] = therapy.sort_values(['event_date'], ascending=True).groupby(['patid', 'event_date'])['code_id'].transform(lambda x: ', '.join(x))

In [ ]:
print(clinical.shape)
print(therapy.shape)

In [13]:
all_vocab = pickle.load(open('../SeqModel/SeqChunks/vocab_1.sav', 'rb'))
temp = pickle.load(open('../SeqModel/SeqChunks/vocab_2.sav', 'rb'))

51314

In [ ]:
all_raw_data = clinical.merge(therapy, how='outer', on=['patid', 'event_date'], suffixes=['_ther', '_clin'], )
clinical = []
therapy = []
all_raw_data = all_raw_data.fillna('NO_CODE')
#extract year, month, day from event date
all_raw_data['day'] = all_raw_data.apply(lambda x: str(x['event_date'].day), axis=1)
all_raw_data['month'] = all_raw_data.apply(lambda x: str(x['event_date'].month), axis=1)
all_raw_data['year'] = all_raw_data.apply(lambda x: str(x['event_date'].year), axis=1)
event_data_seq_all = all_raw_data.sort_values(['event_date'],  ascending=True).groupby('patid').agg({'day': lambda x: x.tolist(),
                                                          'month': lambda x: x.tolist(),
                                                          'year': lambda x: x.tolist()}).reset_index()
all_raw_data['read_code_seq_perdate'] = all_raw_data.apply(lambda x: x.read_code_seq_perdate_clin + ', '+ x.read_code_seq_perdate_ther, axis=1)
all_raw_data=all_raw_data.drop_duplicates(['patid', 'event_date'])
all_raw_data.reset_index(drop=True, inplace=True)
all_raw_data = all_raw_data[['patid', 'event_date', 'read_code_seq_perdate_ther', 'read_code_seq_perdate_clin', 'read_code_seq_perdate']]

In [ ]:
print(all_raw_data.shape)
print(all_raw_data.patid.unique().shape)

In [ ]:
all_raw_data['read_code_seq'] = all_raw_data.sort_values(['event_date'], ascending=True).groupby(['patid'])['read_code_seq_perdate'].transform(lambda x: ', '.join(x))
all_raw_data['read_code_seq_clinical'] = all_raw_data.sort_values(['event_date'], ascending=True).groupby(['patid'])['read_code_seq_perdate_clin'].transform(lambda x: ', '.join(x))
all_raw_data['read_code_seq_therapy'] = all_raw_data.sort_values(['event_date'], ascending=True).groupby(['patid'])['read_code_seq_perdate_ther'].transform(lambda x: ', '.join(x))
all_raw_data=all_raw_data.drop_duplicates(['patid'])
all_raw_data.reset_index(drop=True, inplace=True)
all_raw_data['read_code_seq'] = all_raw_data['read_code_seq'].apply(lambda x: x.strip('""').split(', '))
all_raw_data['length_read_code_seq'] = all_raw_data['read_code_seq'].apply(lambda x: len(x))
all_raw_data = all_raw_data.merge(event_data_seq_all, how='left',  on='patid')


In [ ]:
all_raw_data.length_read_code_seq.describe()

In [ ]:
all_raw_data[all_raw_data.length_read_code_seq>1500].shape

In [ ]:
print(all_raw_data.shape)
print(all_raw_data.patid.unique().shape)

In [ ]:
# all_raw_data = pd.concat([clinical[['patid', 'event_date', 'code_id']],
#            therapy[['patid', 'event_date', 'code_id']]])
# all_raw_data.reset_index(drop=True, inplace=True)

In [ ]:
# all_raw_data.reset_index(drop=True, inplace=True)
# all_raw_data['read_code_seq_perdate'] = all_raw_data.sort_values(['event_date'], ascending=True).groupby(['patid', 'event_date'])['code_id'].transform(lambda x: ', '.join(x))
# all_raw_data['read_code_seq_perdate'] = all_raw_data['read_code_seq_perdate'].apply(lambda x: 'start_visit, ' + x + ', end_visit')
# all_raw_data = all_raw_data.drop_duplicates(subset=['patid', 'event_date']).reset_index(drop=True)
# all_raw_data['read_code_seq'] = all_raw_data.sort_values(['event_date'], ascending=True).groupby(['patid'])['read_code_seq_perdate'].transform(lambda x: ', '.join(x))
# all_raw_data['read_code_seq'] = all_raw_data.sort_values(['event_date'], ascending=False).groupby('patid')['code_id'].transform(lambda x: ', '.join(x))
# all_raw_data = all_raw_data.drop_duplicates(subset=['patid']).reset_index(drop=True)     

In [ ]:
# data_all = data_all[data_all.length_read_code_seq > 10]

In [ ]:
#padding at the beginning of the list

def make_uniform_data(x):
    max_seq = 1500
    if len(x) < max_seq:
        pads = ['PAD'] * (max_seq - len(x))
        return pads + x
    elif len(x) > max_seq:
        x = x[len(x)-max_seq:]
        return x
    else:
        return x
    
#padding at the end of the list
def make_uniform_data_end(x):
    max_seq = 1500
    if len(x) < max_seq:
        pads = ['PAD'] * (max_seq - len(x))
        return x + pads
    elif len(x) > max_seq:
        x = x[len(x)-max_seq:]
        return x
    else:
        return x

In [ ]:
#month indexing
month_vocab = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
month_idx = range(1, len(month_vocab)+1)

month2idx_all = dict(zip(month_vocab, month_idx))
idx2month_all = dict(zip(month_idx, month_vocab))

month2idx_all['PAD'] = 0
# code2idx_all['start_visit'] = 1
# code2idx_all['end_visit'] = 2
idx2month_all[0] = 'PAD'
# idx2code_all[1] = 'start_visit'
# idx2code_all[2] = 'end_visit'
# VOCAB_SIZE = len(month2idx_all)
print('code2idx Size: {}'.format(len(month2idx_all)))
# print('idx2code Size: {}'.format(len(idx2code_all)))

In [ ]:
#vocab and code2idx generation
# vocab_all = all_raw_data.code_id.unique().tolist()
vocab_all = []
for row in all_raw_data[['read_code_seq']].iterrows():
    vocab_all = vocab_all + row[1][0]
vocab_all = list(set(vocab_all))
idx_all = range(1, len(vocab_all)+1)

code2idx_all = dict(zip(vocab_all, idx_all))
idx2code_all = dict(zip(idx_all, vocab_all))

code2idx_all['PAD'] = 0
# code2idx_all['start_visit'] = 1
# code2idx_all['end_visit'] = 2
idx2code_all[0] = 'PAD'
# idx2code_all[1] = 'start_visit'
# idx2code_all[2] = 'end_visit'
VOCAB_SIZE = len(code2idx_all)
print('code2idx Size: {}'.format(len(code2idx_all)))
print('idx2code Size: {}'.format(len(idx2code_all)))

In [ ]:
#put padding at the beginning
all_raw_data['read_code_seq_padded'] = all_raw_data['read_code_seq'].apply(lambda x: make_uniform_data(x))
all_raw_data['read_code_seq_padded_idx'] = all_raw_data['read_code_seq_padded'].apply(lambda x: [code2idx_all.get(key) for key in x])
#as alternative put padding at the end
all_raw_data['read_code_seq_padded_end'] = all_raw_data['read_code_seq'].apply(lambda x: make_uniform_data_end(x))
all_raw_data['read_code_seq_padded_end_idx'] = all_raw_data['read_code_seq_padded_end'].apply(lambda x: [code2idx_all.get(key) for key in x])

all_raw_data = all_raw_data.merge(patient[['patid', 'Country', 'age']], how='left', on='patid')
all_raw_data = all_raw_data.drop_duplicates(subset=['patid']).reset_index(drop=True)     

all_raw_data['month_padded'] = all_raw_data['month'].apply(lambda x: make_uniform_data(x))
all_raw_data['month_padded_idx'] = all_raw_data['month_padded'].apply(lambda x: [month2idx_all.get(key) for key in x])
all_raw_data['month_padded_end'] = all_raw_data['month'].apply(lambda x: make_uniform_data_end(x))
all_raw_data['month_padded_idx_end'] = all_raw_data['month_padded_end'].apply(lambda x: [month2idx_all.get(key) for key in x])

all_raw_data = all_raw_data.merge(outcomes[['patid', '3months', '6months', 
                                   '9months', '12months',]], how='inner', on='patid')

In [ ]:
print(all_raw_data.shape)
print(all_raw_data.patid.unique().shape)

In [ ]:
#SAVE all important materials
pickle.dump(code2idx_all, open('../SeqModel/code2idx_all.sav', 'wb'))
pickle.dump(idx2code_all, open('../SeqModel/idx2code_all.sav', 'wb'))
pickle.dump(month2idx_all, open('../SeqModel/month2idx_all.sav', 'wb'))
pickle.dump(idx2month_all, open('../SeqModel/idx2month_all.sav', 'wb'))
pickle.dump(data_all, open('../SeqModel/data_all.sav', 'wb'))
# data_all.to_csv('../SeqModel/seqData_all.csv', index_label=False, index=False)

# Get all data

In [ ]:
#load clinical and therapy information

#function for sequence padding
def make_uniform_data(x):
    max_seq = 500
    if len(x) < max_seq:
        pads = ['PAD'] * (max_seq - len(x))
        return pads + x
    elif len(x) > max_seq:
        x = x[len(x)-max_seq:]
        return x
    else:
        return x

#padding at the end of the list
def make_uniform_data_end(x):
    max_seq = 500
    if len(x) < max_seq:
        pads = ['PAD'] * (max_seq - len(x))
        return x + pads
    elif len(x) > max_seq:
        x = x[len(x)-max_seq:]
        return x
    else:
        return x
#=========================================================================#

#function to generate the sequence of clinical and therapy data
def process_sequence(clinical_file, therapy_file):
    clinical = pyreadr.read_r(clinical_file)
    clinical = clinical['f_clinical_part']
    therapy_files = [join(path, f) for f in listdir(path) if (isfile(join(path, f))) & ('f_therapy_part' in f)]
    therapy = pyreadr.read_r(therapy_file)
    therapy = therapy['f_therapy_part']

    #data selection 2016&2017
    clinical = clinical.dropna(subset=['code_id'])
    clinical['event_date'] = pd.to_datetime(clinical['event_date'])
    clinical = clinical.loc[(clinical['event_date'] >= '2007-01-01') & (clinical['event_date'] < '2018-01-01')]

    therapy = therapy.dropna(subset=['code_id'])
    therapy['event_date'] = pd.to_datetime(therapy['event_date'])
    therapy = therapy.loc[(therapy['event_date'] >= '2007-01-01') & (therapy['event_date'] < '2018-01-01')]


    #concat 2 data
    temp_all_raw_data = pd.concat([clinical[['patid', 'event_date', 'code_id']],
               therapy[['patid', 'event_date', 'code_id']]])
    temp_all_raw_data.reset_index(drop=True, inplace=True)

    #create vocab list
    temp_vocab = temp_all_raw_data.code_id.unique().tolist()

    #extract year, month, day from event date (if needed)
    temp_all_raw_data['day'] = temp_all_raw_data.apply(lambda x: str(x['event_date'].day), axis=1)
    temp_all_raw_data['month'] = temp_all_raw_data.apply(lambda x: str(x['event_date'].month), axis=1)
    temp_all_raw_data['year'] = temp_all_raw_data.apply(lambda x: str(x['event_date'].year), axis=1)
    temp_event_data_seq_all = temp_all_raw_data.sort_values(['event_date'],  ascending=False).groupby('patid').agg({'day': lambda x: x.tolist(),
                                                          'month': lambda x: x.tolist(),
                                                          'year': lambda x: x.tolist()}).reset_index()

    #checkpoint 1
    print("checkpoint1")
    print(temp_all_raw_data.shape)
    print(temp_all_raw_data.patid.unique().shape)

    #concat all read code per patient per event date (sorted by date, oldest to newest)
    # temp_all_raw_data.reset_index(drop=True, inplace=True)
    # temp_all_raw_data['read_code_seq_perdate'] = temp_all_raw_data.sort_values(['event_date'], ascending=True).groupby(['patid', 'event_date'])['code_id'].transform(lambda x: ', '.join(x))
    # temp_all_raw_data['read_code_seq_perdate'] = temp_all_raw_data['read_code_seq_perdate'].apply(lambda x: 'start_visit, ' + x + ', end_visit')
    # temp_all_raw_data = temp_all_raw_data.drop_duplicates(subset=['patid', 'event_date']).reset_index(drop=True)
    # temp_all_raw_data['read_code_seq'] = temp_all_raw_data.sort_values(['event_date'], ascending=True).groupby(['patid'])['read_code_seq_perdate'].transform(lambda x: ', '.join(x))
    temp_all_raw_data['read_code_seq'] = temp_all_raw_data.sort_values(['event_date'], ascending=False).groupby('patid')['code_id'].transform(lambda x: ', '.join(x))
    temp_all_raw_data = temp_all_raw_data.drop_duplicates(subset=['patid']).reset_index(drop=True)     
    temp_all_raw_data.reset_index(drop=True, inplace=True)

    #checkpoint 2
    print("checkpoint2")
    print(temp_all_raw_data.shape)
    print(temp_all_raw_data.patid.unique().shape)


    #merge sequence data with outcomes
    temp_data_all = temp_all_raw_data[['patid', 'read_code_seq']].merge(temp_event_data_seq_all, how='left',  on='patid')

    #checkpoint 3
    print("checkpoint3")
    print(temp_data_all.shape)
    print(temp_data_all.patid.unique().shape)


    # get sequence length per patient and only keep patient with >10 read codes
    temp_data_all['read_code_seq'] = temp_data_all['read_code_seq'].apply(lambda x: x.strip('""').split(', '))
    temp_data_all['length_read_code_seq'] = temp_data_all['read_code_seq'].apply(lambda x: len(x))
    # temp_data_all = temp_data_all[temp_data_all.length_read_code_seq > 10]

    #checkpoint 4
    print("checkpoint4")
    print(temp_data_all.shape)
    print(temp_data_all.patid.unique().shape)

    #padded data
    temp_data_all['read_code_seq_padded'] = temp_data_all['read_code_seq'].apply(lambda x: make_uniform_data(x))
    temp_data_all['read_code_seq_padded_end'] = temp_data_all['read_code_seq'].apply(lambda x: make_uniform_data_end(x))
    temp_data_all['month_padded'] = temp_data_all['month'].apply(lambda x: make_uniform_data(x))
    temp_data_all['month_padded_end'] = temp_data_all['month'].apply(lambda x: make_uniform_data_end(x))
    
    #checkpoint 5
    print("checkpoint5")
    print(temp_data_all.shape)
    print(temp_data_all.patid.unique().shape)
    
    return temp_data_all, temp_vocab


In [ ]:
path = '../ServerData_13Oct2020/'
clinical_files = [join(path, f) for f in listdir(path) if (isfile(join(path, f))) & ('f_clinical_part' in f)]
therapy_files = [join(path, f) for f in listdir(path) if (isfile(join(path, f))) & ('f_therapy_part' in f)]
vocab_all = []
cols = ['patid', 'read_code_seq', 'day', 'month', 'year', 'length_read_code_seq',
           'read_code_seq_padded']
data_all = pd.DataFrame(columns=cols)

nFiles=len(therapy_files)
batch=10

for i in range(3,5):
    startIndexFile = i*batch
    endIndexFile = startIndexFile+batch
    if endIndexFile > nFiles:
        endIndexFile = nFiles
    print("#######################################################")
    print(startIndexFile, endIndexFile)
    data_chunk = pd.DataFrame(columns=cols)
    vocab_chunk = [] 
    for clinical_file, therapy_file in zip(clinical_files[startIndexFile:endIndexFile], therapy_files[startIndexFile:endIndexFile]):
        print("===================================================")
        print(clinical_file, therapy_file)
        #combine data_all
        temp_data, temp_vocab = process_sequence(clinical_file, therapy_file)
        data_chunk = pd.concat([data_chunk, temp_data])
        data_chunk.reset_index(drop=True, inplace=True)
        vocab_chunk = vocab_chunk + temp_vocab
    data_all = pd.concat([data_all, data_chunk])
    data_all.reset_index(drop=True, inplace=True)
    vocab_all = vocab_all + vocab_chunk
    pickle.dump(vocab_all, open('../SeqModel/vocab_all_big.sav', 'wb'))
    pickle.dump(data_all, open('../SeqModel/data_all_big.sav', 'wb'))
    

In [ ]:
data_all.shape

In [ ]:
data_all.shape

## Combine all chunks

In [ ]:
data_all_big = pickle.load(open('../SeqModel/data_all_big_chunk_0.feather', 'rb'))
vocab_all_big = pickle.load(open('../SeqModel/vocab_all_big_chunk_0.feather', 'rb'))
for i in range(1,2):
    temp_data = pickle.load(open('../SeqModel/data_all_big_chunk_'+str(i)+'.feather', 'rb'))
    temp_vocab = pickle.load(open('../SeqModel/vocab_all_big_chunk_'+str(i)+'.feather', 'rb'))
    data_all_big = pd.concat([data_all_big, temp_data])
    vocab_all_big = vocab_all_big + temp_vocab
    del([temp_data, temp_vocab])
    

In [ ]:
print(data_all_big.shape)
print(len(vocab_all_big))

In [ ]:
#vocab and code2idx generation
# vocab_all_big = vocab_all
vocab_all_big=list(dict.fromkeys(vocab_all_big)) #remove duplicate
idx_all_big = range(1, len(vocab_all_big)+1)
code2idx_all_big = dict(zip(vocab_all_big, idx_all_big))
idx2code_all_big = dict(zip(idx_all_big, vocab_all_big))
code2idx_all_big['PAD'] = 0
# code2idx_all_big['start_visit'] = 1
# code2idx_all_big['end_visit'] = 2
idx2code_all_big[0] = 'PAD'
# idx2code_all_big[1] = 'start_visit'
# idx2code_all_big[2] = 'end_visit'

VOCAB_SIZE_big = len(code2idx_all_big)
print('code2idx Size: {}'.format(len(code2idx_all_big)))
print('idx2code Size: {}'.format(len(idx2code_all_big)))
# print(code2idx)



##month vocab
#month indexing
month_vocab = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
month_idx = range(1, len(month_vocab)+1)

month2idx_all = dict(zip(month_vocab, month_idx))
idx2month_all = dict(zip(month_idx, month_vocab))

month2idx_all['PAD'] = 0
# code2idx_all['start_visit'] = 1
# code2idx_all['end_visit'] = 2
idx2month_all[0] = 'PAD'
# idx2code_all[1] = 'start_visit'
# idx2code_all[2] = 'end_visit'
# VOCAB_SIZE = len(month2idx_all)
print('code2idx Size: {}'.format(len(month2idx_all)))
# print('idx2code Size: {}'.format(len(idx2code_all)))

# padded to index, and merge with patient data (age, Country)
# data_all_big = data_all
data_all_big['read_code_seq_padded_idx'] = data_all_big['read_code_seq_padded'].apply(lambda x: [code2idx_all_big.get(key) for key in x])
data_all_big['read_code_seq_padded_idx_end'] = data_all_big['read_code_seq_padded_end'].apply(lambda x: [code2idx_all_big.get(key) for key in x])
data_all_big['month_padded_idx'] = data_all_big['month_padded'].apply(lambda x: [month2idx_all.get(key) for key in x])
data_all_big['month_padded_idx_end'] = data_all_big['month_padded_end'].apply(lambda x: [month2idx_all.get(key) for key in x])


data_all_big = data_all_big.merge(outcomes[['patid', 'new_3MonthsOutcome', 'new_6MonthsOutcome', 
                                   'new_9MonthsOutcome', 'new_12MonthsOutcome',]], how='inner', on='patid')
data_all_big = data_all_big.merge(patient[['patid', 'Country', 'age']], how='left', on='patid')
data_all_big = data_all_big.drop_duplicates(subset=['patid']).reset_index(drop=True)     

In [ ]:
data_all_big.shape

In [ ]:
data_all_big.patid.unique().shape

In [ ]:
#SAVE all important materials
pickle.dump(code2idx_all_big, open('../SeqModel/code2idx_all_big_06112023_part1.sav', 'wb'))
pickle.dump(idx2code_all_big, open('../SeqModel/idx2code_all_big_06112023_part1.sav', 'wb'))
# pickle.dump(data_all_big, open('../SeqModel/data_all_big_06112023_part1.sav', 'wb'))
data_all_big.to_feather('../SeqModel/data_all_big_06112023_part1.feather')

In [ ]:
data_all_big = pd.read_feather('../SeqModel/data_all_big_06112023_part1.feather')

In [ ]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in list(
                          locals().items())), key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

# END